In [1]:
!ls -liah ./data/health-leading-causes-death-zip-99-13/data/

total 29728
40875098 drwxr-xr-x@ 3 faermanj  ANT\Domain Users   102B 15 May 11:39 .
40875097 drwxr-xr-x@ 5 faermanj  ANT\Domain Users   170B 15 May 11:39 ..
40875099 -rwxr-xr-x@ 1 faermanj  ANT\Domain Users    15M 15 May 09:40 leading_causes_of_death_by_zip_code_1999_2013.csv


In [2]:
dataFile = "./data/health-leading-causes-death-zip-99-13/data/leading_causes_of_death_by_zip_code_1999_2013.csv"

In [3]:
!head {dataFile}

In [4]:
import time
seed = str(int(time.time()))
seed

'1526391316'

In [5]:
bucketName = "ehealthdata" + seed
bucketURL = "s3://" + bucketName
bucketURL

's3://ehealthdata1526391316'

In [6]:
!aws s3 mb {bucketURL}

make_bucket: ehealthdata1526391316


In [7]:
!aws s3 cp {dataFile} {bucketURL}

upload: data/health-leading-causes-death-zip-99-13/data/leading_causes_of_death_by_zip_code_1999_2013.csv to s3://ehealthdata1526391316/leading_causes_of_death_by_zip_code_1999_2013.csv


In [8]:
policy_document = '''{{
  "Version": "2012-10-17",
  "Statement": [{{
    "Effect": "Allow",
    "Action": "s3:*",
    "Resource": [
        "arn:aws:s3:::{bucketName}",
        "arn:aws:s3:::{bucketName}/*"]
    }}
  ]
}}'''
policy_document = policy_document.format(bucketName=bucketName)
print(policy_document)

{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": "s3:*",
    "Resource": [
        "arn:aws:s3:::ehealthdata1526391316",
        "arn:aws:s3:::ehealthdata1526391316/*"]
    }
  ]
}


In [9]:
policy_name = "{}-policy".format(bucketName)
policyArn = ! aws iam create-policy \
    --policy-name {policy_name} \
    --policy-document '{policy_document}' \
    --query Policy.Arn \
    --output text
policyArn = policyArn.s
policyArn

'arn:aws:iam::030555009967:policy/ehealthdata1526391316-policy'

In [10]:
rolename = "{}-role".format(bucketName) 
role_arn = !aws iam create-role \
    --role-name {rolename} \
    --assume-role-policy-document file://glue-trust.json \
    --query Role.Arn \
    --output text
role_arn = role_arn.s
role_arn

'arn:aws:iam::030555009967:role/ehealthdata1526391316-role'

In [11]:
!aws iam attach-role-policy \
    --role-name {rolename} \
    --policy-arn "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
    
!aws iam attach-role-policy \
    --role-name {rolename} \
    --policy-arn {policyArn}
time.sleep(5)

In [12]:
database_name = "{}-gluedb".format(bucketName) 
!aws glue create-database \
    --database-input "Name={database_name}"
database_name

'ehealthdata1526391316-gluedb'

In [13]:
targets = '''
{{
  "S3Targets": [
    {{
      "Path": "{bucketURL}"
    }}
  ]
}}
'''
targets = targets.format(bucketURL=bucketURL)
print(targets)


{
  "S3Targets": [
    {
      "Path": "s3://ehealthdata1526391316"
    }
  ]
}



In [14]:
crawlerName = "crawler-"+seed
!aws glue create-crawler \
    --name  "{crawlerName}" \
    --role '{role_arn}' \
    --database-name "{database_name}" \
    --targets '{targets}'

In [15]:
!aws glue start-crawler \
    --name "{crawlerName}" 
time.sleep(60)

In [16]:
tableName = bucketName
sql = '''
SELECT causes_of_death, sum(count) 
FROM "{}"."{}"
GROUP BY causes_of_death
'''.format(database_name,tableName)
print(sql)


SELECT causes_of_death, sum(count) 
FROM "ehealthdata1526391316-gluedb"."ehealthdata1526391316"
GROUP BY causes_of_death



In [17]:
import boto3

athena = boto3.client('athena')

In [18]:
outputName = "ehealth-workshop-out-" + seed
outputURL = "s3://" + outputName
outputURL

!aws s3 mb '{outputURL}'

make_bucket: ehealth-workshop-out-1526391316


In [19]:
start_query_execution = athena.start_query_execution(
    QueryString=sql,
    QueryExecutionContext={
        'Database': database_name
    },
    ResultConfiguration={
        'OutputLocation': outputURL,        
    }
)
queryId = start_query_execution['QueryExecutionId']
queryId
time.sleep(5)

In [20]:
results = athena.get_query_results(
    QueryExecutionId=queryId
)
results['ResultSet']['Rows']

[{'Data': [{'VarCharValue': 'causes_of_death'}, {'VarCharValue': '_col1'}]},
 {'Data': [{'VarCharValue': 'HYP'}, {'VarCharValue': '41251'}]},
 {'Data': [{'VarCharValue': 'STK'}, {'VarCharValue': '231897'}]},
 {'Data': [{'VarCharValue': 'CAN'}, {'VarCharValue': '822906'}]},
 {'Data': [{'VarCharValue': 'DIA'}, {'VarCharValue': '106960'}]},
 {'Data': [{'VarCharValue': 'HTD'}, {'VarCharValue': '957108'}]},
 {'Data': [{'VarCharValue': 'SUI'}, {'VarCharValue': '52572'}]},
 {'Data': [{'VarCharValue': 'PNF'}, {'VarCharValue': '115926'}]},
 {'Data': [{'VarCharValue': 'CLD'}, {'VarCharValue': '194961'}]},
 {'Data': [{'VarCharValue': 'INJ'}, {'VarCharValue': '157313'}]},
 {'Data': [{'VarCharValue': 'LIV'}, {'VarCharValue': '60526'}]},
 {'Data': [{'VarCharValue': 'HOM'}, {'VarCharValue': '21336'}]},
 {'Data': [{'VarCharValue': 'ALZ'}, {'VarCharValue': '111178'}]},
 {'Data': [{'VarCharValue': 'OTH'}, {'VarCharValue': '637764'}]},
 {'Data': [{'VarCharValue': 'NEP'}, {'VarCharValue': '14338'}]}]